In [2]:
!pip install -q transformers datasets pytesseract pdf2image poppler-utils torchvision
!pip install -q pytorch-lightning


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from pdf2image import convert_from_path
import pytesseract
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from PIL import Image
import os

In [4]:
images = convert_from_path("Input - Construction_Drawings.pdf", dpi=200)

# Save and view first 3 pages
for i, img in enumerate(images[:3]):
    img.save(f"page_{i+1}.png")
    img.show()

In [8]:
def apply_ocr_and_normalize(image):
    width, height = image.size
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    words, boxes = [], []

    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 60 and data['text'][i].strip() != "":
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            
            # Normalize to 0–1000
            x0 = int((x / width) * 1000)
            y0 = int((y / height) * 1000)
            x1 = int(((x + w) / width) * 1000)
            y1 = int(((y + h) / height) * 1000)

            words.append(data['text'][i])
            boxes.append([x0, y0, x1, y1])
    
    return words, boxes

In [9]:
def prepare_input(image_path):
    image = Image.open(image_path).convert("RGB")
    words, boxes = apply_ocr_and_normalize(image)
    
    encoding = processor(image, words, boxes=boxes, return_tensors="pt", truncation=True, padding="max_length")
    return encoding, words, boxes

In [10]:
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=5)  # TODO: fine-tune

encoding, words, boxes = prepare_input("page_1.png")
outputs = model(**encoding)
predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/modeling_utils.py:1735: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [11]:
label_map = {
    0: "O",  # Outside any entity
    1: "B-MATERIAL",
    2: "I-MATERIAL",
    3: "B-SCOPE",
    4: "I-SCOPE",
    # Extend as needed
}

entities = []
for word, pred, box in zip(words, predictions, boxes):
    label = label_map.get(pred, "O")
    if label != "O":
        entities.append({"text": word, "label": label, "box": box})

import pandas as pd
pd.DataFrame(entities)


,text,label,box
0,FOULGER-PRATT,I-SCOPE,"[720, 69, 833, 86]"
1,ISSUED,I-SCOPE,"[756, 203, 804, 220]"
2,BID,I-SCOPE,"[928, 203, 950, 220]"
3,05.28.15,I-SCOPE,"[912, 230, 950, 243]"
4,Copyright,I-SCOPE,"[913, 976, 923, 980]"
5,Perkins+Will,I-SCOPE,"[934, 976, 947, 979]"
6,.,I-SCOPE,"[275, 581, 275, 582]"
7,PERKINS,I-SCOPE,"[849, 900, 955, 918]"
8,+WILL,I-SCOPE,"[877, 933, 954, 951]"
9,ill.com,I-SCOPE,"[159, 967, 184, 976]"


In [3]:
!pip uninstall numpy -y

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6


In [5]:
pip install numpy

  Using cached numpy-2.3.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.3.2-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.3.1 requires altair>=3.2.0, which is not installed.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
pyarrow 15.0.1 requires numpy<2,>=1.16.6, but you have numpy 2.3.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.2 which is incompatible.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.4.0 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.3.2 which is incompatible.
scipy 1.13.0 requires numpy<2.3,>=1.22.4, but you have numpy 2.3.2 which is incompatible.
opencv-python-headless 4.12.0.88 requires 

In [6]:
import pandas as pd
mep = pd.read_excel("MEP - Checklist.xlsx", sheet_name=None)

In [17]:
mep['MEP'].keys()

Index(['Scope of Work', 'Checklist'], dtype='object')

In [23]:
m = pd.DataFrame(mep['MEP'])

In [28]:
m['Checklist'][0:5] # Display first 5 rows

0    Confirm Eyewash counts.  HWH for these if port...
1    Confirm sanitary piping for any recessed walk ...
2              Confirm stone backfill inside buildings
3    Confirms pumps (sumps) triple basins, grease /...
4                   Anything required for guard shack?
Name: Checklist, dtype: object